In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize_2022 import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt
from itertools import permutations, product
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

strategy3 = True

In [3]:
dataObject = DataProcess()

# static measurement jacobian
if strategy3: 
    static_ind = [0,1,2,3,4,5,6,7,8,9,10]
else:
    static_ind = [0,1,2,4,5,6,8,9,10]
static_Nt = 110
static_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(static_Nt)+'_scale.csv')
num_static = len(static_ind)

static_Q = []
for i in static_ind:
    static_Q.append(dataObject.split_jaco(static_jaco, i,static_Nt))

    

dynamic_ind = [3,7,11,12,13]
    
dynamic_Nt = 110
num_dynamic = len(dynamic_ind)
dynamic_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(dynamic_Nt)+'_scale.csv')

dynamic_Q = []
for i in dynamic_ind:
    dynamic_Q.extend(dataObject.split_jaco(dynamic_jaco, i, dynamic_Nt))
    
print("static Q shape:", np.shape(static_Q))
print("dynamic Q shape:", np.shape(dynamic_Q))

jacobian shape: (1540, 5)
jacobian shape: (1540, 5)
static Q shape: (11, 110, 5)
dynamic Q shape: (550, 5)


In [4]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]

num_param = 5
cost = None

num_total = num_static + num_dynamic*dynamic_Nt
print(num_total)

561


In [5]:
calculator = MeasurementOptimizer(static_Q, dynamic_Q, static_Nt, dynamic_Nt, num_param, cost, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(314721, 5, 5)


In [6]:
def check_fim(idx):
    fim = np.asarray(calculator.Q[idx]).T@np.asarray(calculator.Q[idx])
    print(fim)
    print("det:", np.linalg.det(fim))
    print("trace:", np.trace(fim))
    
check_fim(3)
print(calculator.total_no_measure)

[[ 4.91708361e+01  2.73111941e+00 -3.69152159e+00  1.20288318e+02
  -6.86573068e+01]
 [ 2.73111941e+00  1.65082572e-01 -2.68109962e-01  7.03578019e+00
  -3.85432212e+00]
 [-3.69152159e+00 -2.68109962e-01  1.49538628e+00 -9.33843707e+00
   4.82068751e+00]
 [ 1.20288318e+02  7.03578019e+00 -9.33843707e+00  3.07301813e+02
  -1.70094295e+02]
 [-6.86573068e+01 -3.85432212e+00  4.82068751e+00 -1.70094295e+02
   9.64346075e+01]]
det: 0.09544612075380592
trace: 454.56772547997997
561


### Set up grey-box 

In [7]:
from greybox_generalize import LogDetModel

In [8]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [9]:
if strategy3: 
    cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        7000, #ads.z_out (3)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
        7000, #des.z_out (7) 
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]
else:
    cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]

for i in range(dynamic_Nt*num_dynamic):
    cost.append(100)

install_cost_dynamic = [100, 100, 500, 500, 500]


In [10]:
init_y = np.random.rand(num_total-num_static)

update_y = [1]*num_static 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [17]:
def compute_FIM(numRes, FIMSet, cost_list, mixed_integer=False,  obj="A", 
                num_fixed = 9, num_dynamic=5, num_dynamic_t = 100,
                num_dynamic_t_name = None, discretize_time = None, end_time=220,
                fix=False, sparse=False, manual_number=20, budget=100,
               dynamic_install_cost = None, each_manual_number=None):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    
    # dynamic measurements num 
    m.DimDynamic = pyo.Set(initialize=range(num_fixed, num_dynamic+num_fixed))
    m.DimDynamic_t = pyo.Set(initialize=range(num_dynamic_t))
    
    dynamic_time = {}
    for i in range(num_dynamic_t):
        dynamic_time[i] = num_dynamic_t_name[i]
    
    # set up y and cov y 
    def identity(m, a, b):
        return 1 if a==b else 0
    
    def initialize(m, a, b):
        return init_cov_y[a][b]
    
    def NumReshalf_init(m):
        return ((a,b) for a in m.NumRes for b in range(a, numRes))
    
    def DimFIMhalf_init(m):
        return ((a,b) for a in m.DimFIM for b in range(a, len(FIMSet[0])))
    
    m.NumRes_half = pyo.Set(dimen=2, initialize=NumReshalf_init)
    m.DimFIM_half = pyo.Set(dimen=2, initialize=DimFIMhalf_init)

    if mixed_integer:
        if sparse:
            m.cov_y = pyo.Var(m.NumRes_half, initialize=initialize, bounds=(0,1), within=pyo.Binary)
        else:
            m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.Binary)
    else:
        if sparse:
            m.cov_y = pyo.Var(m.NumRes_half, initialize=initialize, bounds=(1E-6,1), within=pyo.NonNegativeReals)
        else:
            m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
        
    def init_fim(m, p, q):
        return fim_prior[p,q]   
    
    # set up FIM
    if sparse:
        m.TotalFIM = pyo.Var(m.DimFIM_half, initialize=init_fim)
    else:
        m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=init_fim)

    # compute FIM
    def eval_fim(m, a, b):
        if a <= b: 
            summi = 0 
            for i in range(numRes):
                for j in range(numRes):
                    if j>=i:
                        summi += m.cov_y[i,j]*FIMSet[i*numRes+j][a][b]
                    else:
                        summi += m.cov_y[j,i]*FIMSet[i*numRes+j][a][b]
            return m.TotalFIM[a,b] == summi
        else:
            return m.TotalFIM[a,b] == m.TotalFIM[b,a]
    
    m.TotalDynamic = pyo.Var(initialize=1)
    
    def total_dynamic(m):
        return m.TotalDynamic==sum(m.cov_y[i,i] for i in range(num_fixed, numRes))
    
    m.manual = pyo.Constraint(rule=total_dynamic)
    #m.manual = pyo.Expression(expr=sum(m.cov_y[i,i] for i in range(num_fixed, numRes)))
            
    # y and conv y constraints
    def y_covy1(m, a, b):
        if b > a:
            return m.cov_y[a, b] <= m.cov_y[a, a]
        else:
            return pyo.Constraint.Skip
        
    def y_covy2(m, a, b):
        if b > a:
            return m.cov_y[a, b] <= m.cov_y[b, b]
        else:
            return pyo.Constraint.Skip
        
    def y_covy3(m, a, b):
        if b>a:
            return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
        else:
            return pyo.Constraint.Skip
        
    def y_covy4(m, a, b):
        if a>b:
            return m.cov_y[a, b] == m.cov_y[a, a]*m.cov_y[b,b]
        else:
            return pyo.Constraint.Skip
        
    def symmetry(m,a,b):
        if a>b:
            return m.cov_y[a,b] == m.cov_y[b,a]
        else:
            return pyo.Constraint.Skip
    
    def cost_compute(m):
        cost = sum(m.cov_y[i,i]*cost_list[i] for i in range(numRes))+sum(m.if_install_dynamic[j]*dynamic_install_cost[j-num_fixed] for j in m.DimDynamic)
        return m.cost == cost

    def cost_limit(m):
        return m.cost <= budget
    
    def total_dynamic_con(m):
        return m.TotalDynamic<=manual_number
    
    def dynamic_fix_yd(m,i,j):
        # map measurement index i to its dynamic_flatten index
        start = num_fixed + (i-num_fixed)*num_dynamic_t+j
        return m.if_install_dynamic[i] >= m.cov_y[j,j]
    
    def dynamic_fix_yd_con2(m,i):
        start = num_fixed + (i-num_fixed)*num_dynamic_t
        end = num_fixed + (i-num_fixed+1)*num_dynamic_t
        return m.if_install_dynamic[i] <= sum(m.cov_y[j,j] for j in range(start, end))
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.TotalFIM[j,j] for j in m.DimFIM)
        return sum_x
    
    
    # add constraints
    if sparse:
        m.TotalFIM_con = pyo.Constraint(m.DimFIM_half, rule=eval_fim)
    else:
        m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        
        if mixed_integer and not sparse:
            m.sym = pyo.Constraint(m.NumRes, m.NumRes, rule=symmetry)
        
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
            
        m.con_manual = pyo.Constraint(rule=total_dynamic_con)
        
        # each manual number smaller than 5 
        if each_manual_number:
            
            for i in range(num_dynamic):
                def dynamic_manual_num(m):
                    start = num_fixed + i*num_dynamic_t
                    end = num_fixed + (i+1)*num_dynamic_t
                    cost = sum(m.cov_y[j,j] for j in range(start, end))
                    return cost <= each_manual_number
                
                con_name = "con"+str(i)
                m.add_component(con_name, pyo.Constraint(expr=dynamic_manual_num))
                
        if discretize_time:
            
            for i in m.DimDynamic:
                for t in range(num_dynamic_t):
                    # end time is an open end of the region, so another constraint needs to be added to include end_time
                    #if dynamic_time[t]+discretize_time <= end_time+0.1*discretize_time:       
                                              
                    def discretizer(m):
                        sumi = 0

                        count = 0 
                        while (count+t<num_dynamic_t) and (dynamic_time[count+t]-dynamic_time[t])<discretize_time:
                            surro_idx = num_fixed + (i-num_fixed)*num_dynamic_t + t + count
                            sumi += m.cov_y[surro_idx, surro_idx]
                            count += 1 

                        return sumi <= 1 

                        #return sum(cov_y[j,j] for j in disc_ind) <= 1 

                    con_name="con_discreti_"+str(i)+str(t)
                    m.add_component(con_name, pyo.Constraint(expr=discretizer))
                    
        # dynamic-cost measurements installaction cost 
        m.if_install_dynamic = pyo.Var(m.DimDynamic, initialize=0, bounds=(0,1.01))
        m.dynamic_cost = pyo.Constraint(m.DimDynamic, m.DimDynamic_t, rule=dynamic_fix_yd)
        m.dynamic_con2 = pyo.Constraint(m.DimDynamic, rule=dynamic_fix_yd_con2)
                
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
     
    # set objective 
    if obj == "A":
        m.Obj = pyo.Objective(rule=ComputeTrace, sense=pyo.maximize)    
    elif obj =="D":
        
        def _model_i(b):
            build_model_external(b)
        m.my_block = pyo.Block(rule=_model_i)
        
        for i in m.DimFIM:
            for j in range(i, len(FIMSet[0])):
                def eq_fim(m):
                    return m.TotalFIM[i,j] == m.my_block.egb.inputs['ele_'+str(i)+'_'+str(j)]
                
                con_name = "con"+str(i)+str(j)
                m.add_component(con_name, pyo.Constraint(expr=eq_fim))

        # add objective to maximize log det
        m.Obj = pyo.Objective(expr=m.my_block.egb.outputs['log_det'], sense=pyo.maximize)

    
    return m 

In [18]:
if strategy3: 
    num_fixed_opt = 11 
    
else:
    num_fixed_opt = 9

num_dynamic_opt = 5

mip_option = False
objective = "D"
sparse_opt = True
fix_opt = False

each_manual_num = 5
total_manual_num = 20
budget_opt = 10000  

discretize_time = 10
num_dynamic_time = np.linspace(2,220,dynamic_Nt)
    
print(num_dynamic_time)

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 
    
print(dynamic_time_dict)

[  2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.  28.
  30.  32.  34.  36.  38.  40.  42.  44.  46.  48.  50.  52.  54.  56.
  58.  60.  62.  64.  66.  68.  70.  72.  74.  76.  78.  80.  82.  84.
  86.  88.  90.  92.  94.  96.  98. 100. 102. 104. 106. 108. 110. 112.
 114. 116. 118. 120. 122. 124. 126. 128. 130. 132. 134. 136. 138. 140.
 142. 144. 146. 148. 150. 152. 154. 156. 158. 160. 162. 164. 166. 168.
 170. 172. 174. 176. 178. 180. 182. 184. 186. 188. 190. 192. 194. 196.
 198. 200. 202. 204. 206. 208. 210. 212. 214. 216. 218. 220.]
{0: 2.0, 1: 4.0, 2: 6.0, 3: 8.0, 4: 10.0, 5: 12.0, 6: 14.0, 7: 16.0, 8: 18.0, 9: 20.0, 10: 22.0, 11: 24.0, 12: 26.0, 13: 28.0, 14: 30.0, 15: 32.0, 16: 34.0, 17: 36.0, 18: 38.0, 19: 40.0, 20: 42.0, 21: 44.0, 22: 46.0, 23: 48.0, 24: 50.0, 25: 52.0, 26: 54.0, 27: 56.0, 28: 58.0, 29: 60.0, 30: 62.0, 31: 64.0, 32: 66.0, 33: 68.0, 34: 70.0, 35: 72.0, 36: 74.0, 37: 76.0, 38: 78.0, 39: 80.0, 40: 82.0, 41: 84.0, 42: 86.0, 43: 88.0, 44: 90.0, 45: 

In [19]:
mod = compute_FIM(num_total, calculator.fim_collection, cost, mixed_integer=mip_option, 
                  num_fixed = num_fixed_opt, num_dynamic = num_dynamic_opt, 
                  num_dynamic_t = dynamic_Nt, num_dynamic_t_name = num_dynamic_time,
                  each_manual_number=each_manual_num, manual_number=total_manual_num,
                  dynamic_install_cost = install_cost_dynamic, discretize_time = discretize_time, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, budget=budget_opt)

if not mip_option and objective=="A":
    solver = pyo.SolverFactory('ipopt')
    solver.options['linear_solver'] = "ma57"
    solver.solve(mod, tee=True)

elif mip_option and objective=="A":
    #solver = pyo.SolverFactory('mindtpy')
    #results = solver.solve(mod, mip_solver='gurobi_persistent', nlp_solver='ipopt', 
    #                       time_limit=36000, iteration_limit=100000,
    #                       tee=True, integer_tolerance=0.5)
    solver = pyo.SolverFactory('gurobi', solver_io="python")
    solver.options['mipgap'] = 0.1
    results = solver.solve(mod, tee=True)
    
elif objective=="D":  
    solver = pyo.SolverFactory('cyipopt')
    solver.config.options['hessian_approximation'] = 'limited-memory' 
    additional_options={'max_iter':3000}
    
    for k,v in additional_options.items():
        solver.config.options[k] = v
    results = solver.solve(mod, tee=True)


 Consider M =
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
   logdet =  0.0 


 Consider M =
 [[ 0.999752    0.01268122 -0.01028599 -0.00200062 -0.00404606]
 [ 0.01268122  1.0011701  -0.01074376 -0.00913847  0.00385688]
 [-0.01028599 -0.01074376  0.99927202  0.00107636 -0.00699082]
 [-0.00200062 -0.00913847  0.00107636  1.00126446 -0.00104839]
 [-0.00404606  0.00385688 -0.00699082 -0.00104839  0.99836832]]
   logdet =  -0.0007249272329535688 


 Consider M =
 [[ 1.00240445  0.0154026  -0.01309348 -0.00224828 -0.00259839]
 [ 0.0154026   1.00139664 -0.01312094 -0.00685113  0.00475089]
 [-0.01309348 -0.01312094  1.00263747  0.00223885 -0.01043724]
 [-0.00224828 -0.00685113  0.00223885  1.00905195 -0.00151725]
 [-0.00259839  0.00475089 -0.01043724 -0.00151725  0.99432734]]
   logdet =  0.00898418641204093 


 Consider M =
 [[ 1.21316163  0.00940912 -0.07219826 -0.14295337 -0.09688671]
 [ 0.00940912  1.00949315  0.00210973 -0.28749278  0.02590


 Consider M =
 [[ 247.50803909   -9.17455948 -192.18459287  142.0347266     8.53722642]
 [  -9.17455948    1.49377725    6.08453041  -11.85980668    3.01861695]
 [-192.18459287    6.08453041  179.2560911  -168.55603313  -24.35004192]
 [ 142.0347266   -11.85980668 -168.55603313   20.15623923   43.27847854]
 [   8.53722642    3.01861695  -24.35004192   43.27847854   24.09931345]]
   logdet =  16.73956066825736 


 Consider M =
 [[ 267.17106136   -9.42231926 -234.99182606  224.86541658   27.30321465]
 [  -9.42231926    1.40394539    6.35813767  -12.95592667    2.5870686 ]
 [-234.99182606    6.35813767  238.46777532 -311.31936967  -45.06598747]
 [ 224.86541658  -12.95592667 -311.31936967  328.13583363   95.77045499]
 [  27.30321465    2.5870686   -45.06598747   95.77045499   35.12228815]]
   logdet =  17.710837183997036 


 Consider M =
 [[ 295.36083073   -9.81250119 -237.69385315  261.35453727    7.64054669]
 [  -9.81250119    1.18844484    5.82393952  -13.65667627    4.06972133]
 [-237.


 Consider M =
 [[ 6.80578334e+02 -5.15295953e+00 -4.57471405e+02  1.20006670e+03
  -8.02650468e+01]
 [-5.15295953e+00  1.00429158e+00  4.81722891e+00  9.08444375e-01
   6.03863541e+00]
 [-4.57471405e+02  4.81722891e+00  4.15910086e+02 -9.81144218e+02
  -2.98951772e+01]
 [ 1.20006670e+03  9.08444375e-01 -9.81144218e+02  3.36199822e+03
   2.73326330e+01]
 [-8.02650468e+01  6.03863541e+00 -2.98951772e+01  2.73326330e+01
   8.05423868e+01]]
   logdet =  21.565467356632134 


 Consider M =
 [[ 1.46859731e+03 -6.62162923e+00 -1.06596374e+03  2.81215216e+03
  -1.29382410e+02]
 [-6.62162923e+00  9.90979477e-01  6.62906731e+00 -1.74129648e+00
   4.23427941e+00]
 [-1.06596374e+03  6.62906731e+00  8.84689034e+02 -2.17143112e+03
   1.08731998e+00]
 [ 2.81215216e+03 -1.74129648e+00 -2.17143112e+03  6.67637228e+03
  -1.20598931e+02]
 [-1.29382410e+02  4.23427941e+00  1.08731998e+00 -1.20598931e+02
   1.03183156e+02]]
   logdet =  21.667240083026766 


 Consider M =
 [[ 1.51183017e+03 -6.97652362e+0


 Consider M =
 [[ 2.80307303e+03  1.10125923e+01 -2.01857705e+03  5.94298710e+03
  -2.42919039e+02]
 [ 1.10125923e+01  9.10142455e-01 -5.50331308e+00  6.57738654e+01
   5.38519969e+00]
 [-2.01857705e+03 -5.50331308e+00  1.60146323e+03 -4.46036104e+03
   5.96956565e+01]
 [ 5.94298710e+03  6.57738654e+01 -4.46036104e+03  1.53440563e+04
  -4.33450592e+02]
 [-2.42919039e+02  5.38519969e+00  5.96956565e+01 -4.33450592e+02
   1.38774817e+02]]
   logdet =  25.224032677333156 


 Consider M =
 [[ 2.80325966e+03  1.08261199e+01 -2.01834956e+03  5.94282474e+03
  -2.43204369e+02]
 [ 1.08261199e+01  9.41063909e-01 -5.53444580e+00  6.56513053e+01
   5.72928927e+00]
 [-2.01834956e+03 -5.53444580e+00  1.60183167e+03 -4.46021900e+03
   5.91529968e+01]
 [ 5.94282474e+03  6.56513053e+01 -4.46021900e+03  1.53441339e+04
  -4.33519304e+02]
 [-2.43204369e+02  5.72928927e+00  5.91529968e+01 -4.33519304e+02
   1.38905445e+02]]
   logdet =  25.263395641922813 


 Consider M =
 [[ 2.80326019e+03  1.08405412e+0


 Consider M =
 [[ 2.76287056e+03  1.04336432e+01 -1.96842091e+03  5.83030021e+03
  -2.53551790e+02]
 [ 1.04336432e+01  9.46023116e-01 -5.77941736e+00  6.93708316e+01
   6.02979528e+00]
 [-1.96842091e+03 -5.77941736e+00  1.57203423e+03 -4.40110264e+03
   4.80554514e+01]
 [ 5.83030021e+03  6.93708316e+01 -4.40110264e+03  1.54827305e+04
  -4.31430426e+02]
 [-2.53551790e+02  6.02979528e+00  4.80554514e+01 -4.31430426e+02
   1.58589163e+02]]
   logdet =  25.575786499615585 


 Consider M =
 [[ 2.75267628e+03  1.04690923e+01 -1.97053756e+03  5.82369715e+03
  -2.46678642e+02]
 [ 1.04690923e+01  9.54031047e-01 -5.75152180e+00  6.95736494e+01
   5.91731626e+00]
 [-1.97053756e+03 -5.75152180e+00  1.58053139e+03 -4.42434747e+03
   4.47786189e+01]
 [ 5.82369715e+03  6.95736494e+01 -4.42434747e+03  1.55320800e+04
  -4.13865389e+02]
 [-2.46678642e+02  5.91731626e+00  4.47786189e+01 -4.13865389e+02
   1.57448491e+02]]
   logdet =  25.564593878595034 


 Consider M =
 [[ 2.76418475e+03  1.07335409e+0

...


 Consider M =
 [[ 3.00385158e+03  1.47083481e+01 -2.23222675e+03  6.71118926e+03
  -2.59448280e+02]
 [ 1.47083481e+01  8.91222664e-01 -2.98812413e+00  7.81570672e+01
  -2.23561918e+00]
 [-2.23222675e+03 -2.98812413e+00  1.84138241e+03 -5.20161025e+03
   3.87637200e+01]
 [ 6.71118926e+03  7.81570672e+01 -5.20161025e+03  1.83638277e+04
  -5.55704774e+02]
 [-2.59448280e+02 -2.23561918e+00  3.87637200e+01 -5.55704774e+02
   2.11968112e+02]]
   logdet =  25.834090333010153 


 Consider M =
 [[ 3.00506406e+03  1.47350627e+01 -2.23261171e+03  6.71320203e+03
  -2.59143962e+02]
 [ 1.47350627e+01  8.95754263e-01 -2.99403149e+00  7.82963561e+01
  -2.24118234e+00]
 [-2.23261171e+03 -2.99403149e+00  1.84120184e+03 -5.20164548e+03
   3.89072390e+01]
 [ 6.71320203e+03  7.82963561e+01 -5.20164548e+03  1.83671368e+04
  -5.55282625e+02]
 [-2.59143962e+02 -2.24118234e+00  3.89072390e+01 -5.55282625e+02
   2.10975483e+02]]
   logdet =  25.830569259933718 


 Consider M =
 [[ 3.00633423e+03  1.47245721e+0


 Consider M =
 [[ 3.07507471e+03  1.53463828e+01 -2.29158311e+03  6.92100036e+03
  -2.70469428e+02]
 [ 1.53463828e+01  9.85662905e-01 -3.33359620e+00  8.08154952e+01
  -2.54110719e+00]
 [-2.29158311e+03 -3.33359620e+00  1.89268085e+03 -5.37850068e+03
   4.54945370e+01]
 [ 6.92100036e+03  8.08154952e+01 -5.37850068e+03  1.90072918e+04
  -5.77478114e+02]
 [-2.70469428e+02 -2.54110719e+00  4.54945370e+01 -5.77478114e+02
   2.18445159e+02]]
   logdet =  25.85909318849209 


 Consider M =
 [[ 3.08050777e+03  1.53554692e+01 -2.29483983e+03  6.93663756e+03
  -2.71845109e+02]
 [ 1.53554692e+01  9.90523442e-01 -3.29159626e+00  8.09270339e+01
  -2.57705646e+00]
 [-2.29483983e+03 -3.29159626e+00  1.89546506e+03 -5.38767315e+03
   4.53673490e+01]
 [ 6.93663756e+03  8.09270339e+01 -5.38767315e+03  1.90528282e+04
  -5.81834082e+02]
 [-2.71845109e+02 -2.57705646e+00  4.53673490e+01 -5.81834082e+02
   2.19619402e+02]]
   logdet =  25.870228926068428 


 Consider M =
 [[ 3.08045846e+03  1.53530841e+01


 Consider M =
 [[ 3.14856523e+03  1.84584238e+01 -2.32333526e+03  7.12991498e+03
  -2.86412008e+02]
 [ 1.84584238e+01  1.08970298e+00 -3.91648389e+00  8.76857422e+01
  -4.03445168e+00]
 [-2.32333526e+03 -3.91648389e+00  1.89783665e+03 -5.45607558e+03
   5.44090013e+01]
 [ 7.12991498e+03  8.76857422e+01 -5.45607558e+03  1.95295519e+04
  -6.31619015e+02]
 [-2.86412008e+02 -4.03445168e+00  5.44090013e+01 -6.31619015e+02
   2.27742583e+02]]
   logdet =  25.990266575212065 


 Consider M =
 [[ 3.14828163e+03  1.84682261e+01 -2.32336233e+03  7.13007106e+03
  -2.86303630e+02]
 [ 1.84682261e+01  1.08883173e+00 -3.93223918e+00  8.76957665e+01
  -4.02589565e+00]
 [-2.32336233e+03 -3.93223918e+00  1.89794693e+03 -5.45683232e+03
   5.44060923e+01]
 [ 7.13007106e+03  8.76957665e+01 -5.45683232e+03  1.95317462e+04
  -6.31302761e+02]
 [-2.86303630e+02 -4.02589565e+00  5.44060923e+01 -6.31302761e+02
   2.27759665e+02]]
   logdet =  25.990453915409724 


 Consider M =
 [[ 3.14728785e+03  1.86129416e+0


 Consider M =
 [[ 3.18449347e+03  2.05106898e+01 -2.33040769e+03  7.23695946e+03
  -2.96624159e+02]
 [ 2.05106898e+01  1.12352228e+00 -5.16175294e+00  9.26644496e+01
  -4.13012299e+00]
 [-2.33040769e+03 -5.16175294e+00  1.89273492e+03 -5.49427791e+03
   5.63124836e+01]
 [ 7.23695946e+03  9.26644496e+01 -5.49427791e+03  1.98304617e+04
  -6.46360311e+02]
 [-2.96624159e+02 -4.13012299e+00  5.63124836e+01 -6.46360311e+02
   2.36491200e+02]]
   logdet =  26.072091198077537 


 Consider M =
 [[ 3.18125382e+03  2.05102736e+01 -2.32756210e+03  7.22797716e+03
  -2.96636876e+02]
 [ 2.05102736e+01  1.12215854e+00 -5.14055026e+00  9.26203475e+01
  -4.16581081e+00]
 [-2.32756210e+03 -5.14055026e+00  1.89049952e+03 -5.48646038e+03
   5.57612837e+01]
 [ 7.22797716e+03  9.26203475e+01 -5.48646038e+03  1.98045724e+04
  -6.46371184e+02]
 [-2.96636876e+02 -4.16581081e+00  5.57612837e+01 -6.46371184e+02
   2.37555694e+02]]
   logdet =  26.07505392389472 


 Consider M =
 [[ 3.18003013e+03  2.05083749e+01


 Consider M =
 [[ 3.17855242e+03  2.05330656e+01 -2.32456934e+03  7.21932362e+03
  -2.97211975e+02]
 [ 2.05330656e+01  1.12141746e+00 -5.11008786e+00  9.25871795e+01
  -4.21626424e+00]
 [-2.32456934e+03 -5.11008786e+00  1.88826242e+03 -5.47853405e+03
   5.51609778e+01]
 [ 7.21932362e+03  9.25871795e+01 -5.47853405e+03  1.97793388e+04
  -6.47172360e+02]
 [-2.97211975e+02 -4.21626424e+00  5.51609778e+01 -6.47172360e+02
   2.39426660e+02]]
   logdet =  26.085402648442273 


 Consider M =
 [[ 3.17855832e+03  2.05330927e+01 -2.32457377e+03  7.21933872e+03
  -2.97212866e+02]
 [ 2.05330927e+01  1.12142000e+00 -5.11011362e+00  9.25872680e+01
  -4.21623497e+00]
 [-2.32457377e+03 -5.11011362e+00  1.88826602e+03 -5.47854652e+03
   5.51617403e+01]
 [ 7.21933872e+03  9.25872680e+01 -5.47854652e+03  1.97793819e+04
  -6.47173649e+02]
 [-2.97212866e+02 -4.21623497e+00  5.51617403e+01 -6.47173649e+02
   2.39426378e+02]]
   logdet =  26.085406553749102 


 Consider M =
 [[ 3.17855614e+03  2.05330777e+0

In [27]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 3.17855533e+03  2.05330722e+01 -2.32457144e+03  7.21933073e+03
  -2.97212327e+02]
 [ 2.05330722e+01  1.12141774e+00 -5.11009611e+00  9.25871947e+01
  -4.21623311e+00]
 [-2.32457144e+03 -5.11009611e+00  1.88826420e+03 -5.47854012e+03
   5.51612648e+01]
 [ 7.21933073e+03  9.25871947e+01 -5.47854012e+03  1.97793596e+04
  -6.47172563e+02]
 [-2.97212327e+02 -4.21623311e+00  5.51612648e+01 -6.47172563e+02
   2.39426334e+02]]
trace: 25086.726830225827
det: -213180817418.5745
[ 2.40942641e+04  6.55526459e+02  3.11745200e+02  2.68062151e+01
 -1.61513548e+00]
Pyomo OF: 26.085406550605807
Log_det: nan


/home/parallels/anaconda3/envs/pyomo/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


In [23]:
# problem size calculator
if True:
    nr = num_total
    npa = 5
    nd = 5
    nd_time = 110
    num_var = (nr*nr+nr)/2 + (npa*npa+npa)/2 + nd +2
    num_equal = (npa*npa+npa)/2+2
    num_ineq = 1.5*(nr**2-nr)+2+nd+2*nd*nd_time

    print("# variables:", num_var)
    print("# equality constraints:", num_equal)
    print("# inequality constraints:", num_ineq)

# variables: 157663.0
# equality constraints: 17.0
# inequality constraints: 472347.0


In [24]:
ans_y = np.zeros((num_total,num_total))

pyomo_y = np.zeros((num_total, num_total))

for i in range(num_total):
    for j in range(i, num_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = ans_y[j,i] = cov 
        
print("Cost for this stragy:", pyo.value(mod.cost))
print("Cost if no budget:", sum(cost))



#print(pyo.value(mod.cov_y[9,9]))
#print(pyo.value(mod.if_install_dynamic[9]))

#for i in range(9,119):
#    print(pyo.value(mod.cov_y[i,i]))
#print(sum(pyo.value(mod.cov_y[i,i]) for i in range(9,119)))

Cost for this stragy: 10000.000099923798
Cost if no budget: 77000


In [25]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

[[0.5  0.   0.5  ... 0.   0.   0.  ]
 [0.   0.41 0.41 ... 0.   0.   0.  ]
 [0.5  0.41 0.5  ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [ ]:
import pickle

store = False

if store:
    file = open('Mar20_A_110_8000_strategy3_mip', 'wb')

    pickle.dump(ans_y, file)

    file.close()

In [26]:
for c in range(num_static):
    print(static_name[c], ": ", ans_y[c,c])

sol_y = np.asarray([ans_y[i,i] for i in range(num_static, num_total)])

sol_y = np.reshape(sol_y, (num_dynamic, dynamic_Nt))
np.around(sol_y)

for r in range(len(sol_y)):
    #print(dynamic_name[r], ": ", sol_y[r])
    print(dynamic_name[r], "(time points [min]):")
    for i, t in enumerate(sol_y[r]):
        if t>0.5:
            print(dynamic_time_dict[i])
    
# verify cost 
real_cost = 0 
for i in range(num_total):
    real_cost += ans_y[i,i]*cost[i]
    
dynamic_inst = [0]*num_dynamic
for j in range(num_dynamic):
    for t in range(dynamic_Nt):
        if sol_y[j,t]>0.99:
            dynamic_inst[j] = 1 
            
real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
print(dynamic_inst)
print(real_cost)
    


Ads.gas_inlet.F :  0.5
Ads.gas_outlet.F :  0.41
Ads.gas_outlet.T :  0.5
Ads.gas_outlet.z("CO2") :  0.41
Des.gas_inlet.F :  0.59
Des.gas_outlet.F :  0.59
Des.gas_outlet.T :  0.59
Des.gas_outlet.z("CO2") :  0.17
Ads.T_g.Value(19,10) :  0.59
Ads.T_g.Value(23,10) :  0.5
Ads.T_g.Value(28,10) :  0.5
Ads.gas_outlet.z("CO2") (time points [min]):
110.0
Des.gas_outlet.z("CO2") (time points [min]):
182.0
Ads.z("CO2",19,10) (time points [min]):
214.0
Ads.z("CO2",23,10) (time points [min]):
214.0
Ads.z("CO2",28,10) (time points [min]):
86.0
104.0
114.0
124.0
174.0
[0, 0, 0, 0, 1]
9519.0


In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
#measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
#measure_name = ['Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
measure_name = ['a', 'b', 'c', 'd', 'e']

#time_set = ['500', '1000', '1500', '2000', '2200']
time_set = np.linspace(20, 2200, number_t)
time_set = [str(i) for i in time_set]
print(len(time_set), len(measure_name))
print(np.shape(sol_y))

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, time_set, measure_name, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri):

    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    #plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(A-optimality)', fontweight="bold")
    
    plt.show()
    
    
    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(D-optimality)', fontweight="bold")
    
    plt.show()

In [ ]:
a_set = [65693.94, 66574.63, 67474.485, 67608.817]
d_set = [np.exp(30.13), np.exp(30.719), np.exp(31.1205), np.exp(31.2586)]

plot_data([3000,4000,5000,6400], a_set, d_set)